In [40]:
#imports
#%cd '/content/drive/My Drive/Spaceapps-2020/Model'
#import import_ipynb
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from preprocess import preprocess
from imblearn.over_sampling import RandomOverSampler

In [41]:
#checking for gpu stats
#LITE
#!nvidia-smi
X,y,coords = preprocess('/home/l12-o-0-554/Desktop/SpaceApps/notebooks/spaceapps-2020/MOD14.A2020031.0910.006.2020031142922.hdf',
                       '/home/l12-o-0-554/Desktop/SpaceApps/notebooks/spaceapps-2020/MOD04_L2.A2020031.0910.061.2020031193711.hdf',
                       '/home/l12-o-0-554/Desktop/SpaceApps/notebooks/spaceapps-2020/MOD11_L2.A2020031.0910.006.2020032074934.hdf')

In [42]:
print(X.shape,y.shape)
oversample = RandomOverSampler(sampling_strategy='minority')
X_o,y_o = oversample.fit_resample(X,y)

(27405, 8) (27405,)


In [43]:
#activating gpu

#tf.config.experimental.set_memory_growth(tf.config.experimental.list_physical_devices('GPU')[0], True)

In [44]:
#the dropout is primarily incase we pick a completely wrong param and the model hopefully learns to ignore it
#update the input shape in the first layer to change the number of input params, and the number of units in the last layer for the number of outputs classes we have

model= Sequential([
    Dense(units=32, input_shape=(8,), activation='relu'),
    Dropout(0.2),
    Dense(units=64, activation='relu'),
    Dropout(0.2),
    Dense(units=128, activation='relu'),
    Dropout(0.2),
    Dense(units=1, activation='sigmoid')
])

In [45]:
#self explanatory

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 32)                288       
_________________________________________________________________
dropout_12 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_17 (Dense)             (None, 64)                2112      
_________________________________________________________________
dropout_13 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_18 (Dense)             (None, 128)               8320      
_________________________________________________________________
dropout_14 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 1)                

In [46]:
#don't change unless you're sure of what you're doing

model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [47]:
#x gets the input dataset, y is the labels, validation split is for how much you wanna segment for validation, epochs is number of passes through the data
#if we use a different dataloader which also incorporates batch size, then we will have to modify the commented out fields as well

model.fit(x=X_o ,y=y_o, validation_split=0.4 , batch_size=3000 , epochs=100 , verbose=1)#, steps_per_epoch= , validation_steps= )

Epoch 1/100
11/11 [==============================] - 1s 50ms/step - loss: 298.5720 - accuracy: 0.7459 - val_loss: 645.0212 - val_accuracy: 0.0604
Epoch 2/100
11/11 [==============================] - 0s 30ms/step - loss: 180.8867 - accuracy: 0.7418 - val_loss: 220.3331 - val_accuracy: 0.0000e+00
Epoch 3/100
11/11 [==============================] - 0s 36ms/step - loss: 129.0182 - accuracy: 0.7819 - val_loss: 100.3930 - val_accuracy: 0.0000e+00
Epoch 4/100
11/11 [==============================] - 0s 29ms/step - loss: 102.0003 - accuracy: 0.7592 - val_loss: 74.4829 - val_accuracy: 0.0000e+00
Epoch 5/100
11/11 [==============================] - 0s 26ms/step - loss: 81.1637 - accuracy: 0.7661 - val_loss: 40.3060 - val_accuracy: 0.6465
Epoch 6/100
11/11 [==============================] - 0s 31ms/step - loss: 68.8923 - accuracy: 0.7633 - val_loss: 79.3465 - val_accuracy: 0.0000e+00
Epoch 7/100
11/11 [==============================] - 0s 30ms/step - loss: 56.2029 - accuracy: 0.7742 - val_loss: 

Epoch 57/100
11/11 [==============================] - 0s 37ms/step - loss: 0.6722 - accuracy: 0.8241 - val_loss: 1.2104 - val_accuracy: 0.0000e+00
Epoch 58/100
11/11 [==============================] - 0s 37ms/step - loss: 0.6626 - accuracy: 0.8258 - val_loss: 1.0753 - val_accuracy: 0.0000e+00
Epoch 59/100
11/11 [==============================] - 0s 39ms/step - loss: 0.6280 - accuracy: 0.8221 - val_loss: 1.1580 - val_accuracy: 0.0000e+00
Epoch 60/100
11/11 [==============================] - 0s 32ms/step - loss: 0.6174 - accuracy: 0.8217 - val_loss: 1.2154 - val_accuracy: 0.0000e+00
Epoch 61/100
11/11 [==============================] - 0s 41ms/step - loss: 0.6285 - accuracy: 0.8240 - val_loss: 1.0716 - val_accuracy: 0.0000e+00
Epoch 62/100
11/11 [==============================] - 0s 38ms/step - loss: 0.5909 - accuracy: 0.8268 - val_loss: 1.0670 - val_accuracy: 0.0000e+00
Epoch 63/100
11/11 [==============================] - 0s 43ms/step - loss: 0.5826 - accuracy: 0.8267 - val_loss: 1.181

In [48]:
predictions=model.predict(x=X , verbose=0)#, steps= )

In [49]:
print(predictions)
model.evaluate(X,y)

[[0.1099731 ]
 [0.10026509]
 [0.03688163]
 ...
 [0.03314853]
 [0.03306776]
 [0.03319918]]
857/857 [==============================] - 2s 2ms/step - loss: 0.2353 - accuracy: 0.9747


[0.23534339666366577, 0.9746761322021484]

In [20]:
model.save("spacia.h5")